In [ ]:
import gpt2 as nn
import jax.numpy as jnp
import jax
import tiktoken
import numpy as np
from importlib import reload
reload(nn)

<module 'gpt2' from '/content/gpt2.py'>

In [ ]:
model = nn.GPT2(nn.Config())
params = model.from_pretrained('gpt2')

# model_blank = nn.GPT2(nn.Config())

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


flax_model.msgpack:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
#inputs = jnp.array(inputs['input_ids'])
enc = tiktoken.get_encoding('gpt2')
key = jax.random.PRNGKey(42069)
key, gen_key = jax.random.split(key)
start = "Hello, I'm a language model,"
inputs = jnp.array(enc.encode(start)).reshape(1,-1)
out = model.generate_batch(key, params, inputs, 1000)
#jit_gen = jax.jit(lambda params, gen_key, inputs, num_new_tokens: model.generate_batch(gen_key, params, inputs, num_new_tokens), static_argnums=(3,))

KeyboardInterrupt: 

In [ ]:
#out = jit_gen(params, gen_key, inputs, 1000)

In [ ]:
def _gen_step_hf(model_fwd, carry, rng):
    window, params = carry  # window: (B, block_size)
    logits = model_fwd(params, window).logits
    next_token = jax.random.categorical(rng, logits[:, -1, :])  # (B,)
    new_window = jnp.concatenate([window[:, 1:], next_token[:,None]], axis=1) #add next_token[:,None] here instead
    return (new_window, params), next_token #return next_token without adding extra dimension


def generate_batch_hf(key, model_fwd, params, init_idx, max_new_tokens: int):
    """Much faster but slow on CPU: Use on GPU/TPU"""
    B, T0 = init_idx.shape
    assert T0 <= nn.Config.block_size, f"Context length must be ≤ block_size ({nn.Config.block_size}), got {T0}"

    # left-pad init_idx up to block_size so our carry window is fixed-size
    pad_len     = nn.Config.block_size - T0
    init_window = jnp.pad(init_idx, ((0,0), (pad_len,0)), constant_values=0)  # (B, block_size)

    # split RNG into one key per token
    keys = jax.random.split(key, max_new_tokens)

    # run the scan
    (final_window, final_params), new_tokens = jax.lax.scan(
        lambda carry, rng: _gen_step_hf(model_fwd, carry, rng),
        (init_window, params),  # initial carry
        keys             # scan over these RNGs
    )
    # new_tokens: (max_new_tokens, B)

    context = init_window[:, pad_len:]               # (B, T0)
    #gen_seq = jnp.transpose(new_tokens, (1,0))       # (B, max_new_tokens)
    gen_seq = new_tokens.reshape(new_tokens.shape[1], new_tokens.shape[0]) # Reshape new_tokens to (B, max_new_tokens)
    #gen_seq = new_tokens.squeeze()
    full_seq = jnp.concatenate([context, gen_seq], axis=1)  # (B, T0 + max_new_tokens)

    return full_seq

In [ ]:
from transformers import AutoTokenizer, FlaxGPT2LMHeadModel, GPT2Config

hf_model = FlaxGPT2LMHeadModel.from_pretrained("openai-community/gpt2")
hf_params = hf_model.params


In [ ]:
hf_out = generate_batch_hf(gen_key, lambda params, input: hf_model(input), hf_params, inputs, 100)
hf_output_text = enc.decode(hf_out[0])

In [ ]:
out = model.generate_batch(gen_key, params, inputs, 100)
my_model_output_text = enc.decode(out[0])

In [ ]:
assert hf_output_text == my_model_output_text, "Weights have been loaded incorrectly"